In [1]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `K` was defined in the following phases: ['FCC_A1', 'FCC_L12'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `S` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `P` was defined in the following phases: ['FCC_A1', 'FCC_4SL', 'HCP_A3', 'HCP_4SL'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(

In [2]:
# constants:
P = 101325

# Компоненты и фазы

In [3]:
import random

random.seed(10)
all_components = list(db.elements)
all_phases =  list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


Количество фаз =2023
Количество компонент =80
['AG', 'AL', 'AM', 'AS', 'AU', 'B', 'BA', 'BE', 'BI', 'C', 'CA', 'CD', 'CE', 'CO', 'CR', 'CS', 'CU', 'DY', 'ER', 'EU', 'FE', 'GA', 'GD', 'GE', 'H', 'HF', 'HG', 'HO', 'IN', 'IR', 'K', 'LA', 'LI', 'LU', 'MG', 'MN', 'MO', 'N', 'NA', 'NB', 'ND', 'NI', 'NP', 'O', 'OS', 'P', 'PA', 'PB', 'PD', 'PR', 'PT', 'PU', 'RB', 'RE', 'RH', 'RU', 'S', 'SB', 'SC', 'SE', 'SI', 'SM', 'SN', 'SR', 'TA', 'TB', 'TC', 'TE', 'TH', 'TI', 'TL', 'TM', 'U', 'V', 'VA', 'W', 'Y', 'YB', 'ZN', 'ZR']
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']


# Бинарные фазы для компонент

In [4]:
counter = 0 
possible_phases = []

for phase in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints    
    found_sublattices = 0
    for sublattice_c in phase[1].constituents:
        found = False
        for comp in components:
            if len([species.name for species in sublattice_c if species.name == comp]) > 0:
                found = True
                break
        if found:
            found_sublattices += 1
        else:
            break
    
    if found_sublattices == len(phase[1].constituents):
        possible_phases.append(phase[0])
        
possible_phases.sort()
print(possible_phases)

['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 'RHOMBO_C19', 'SIGMA', 'TETRAGONAL_A6',

In [5]:
import numpy as np

def getPhases(int_mask, all_phases = all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]
def phasesToMask(phases, all_phases = all_phases):
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)
def phaseToMask(phase, all_phases = all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

phases_mask = phasesToMask(possible_phases)
print(possible_phases == getPhases(phases_mask, all_phases))

True


# Расчет фазовых равновесий

In [7]:
skip_cond_component = 'V'
temp = 1200
conditions = {v.T:temp,v.P:P}
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']
# TODO: make random amount of component
amount = 1.0/len(components)

for comp in components:
    if comp != skip_cond_component:
        conditions[v.X(comp)] = amount

calc_components = components
calc_components.append('VA')
print(conditions)
print(calc_components)
eq = equilibrium(db,calc_components,getPhases(phases_mask),conditions, verbose=False)
# gibbs_energy = eq.GM

{T: 1200, P: 101325, X_AU: 0.125, X_RH: 0.125, X_SM: 0.125, X_ZR: 0.125, X_AL: 0.125, X_HG: 0.125, X_SE: 0.125}
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE', 'VA']


In [8]:
def flatten_array(arr):
    result = []
    for item in arr.flat:
        if isinstance(item, np.ndarray):
            result.extend(flatten_array(item))
        else:
            result.append(item)
    return result

In [9]:
eq_phases = flatten_array(np.array(eq.Phase))
print(eq_phases)
eq_phases_bin = [phaseToMask(phase) for phase in eq_phases]
print([getPhases(phase) for phase in eq_phases_bin])

['AUZR', 'LIQUID', 'AL2SM', '', '', '', '', '', '']
[['AUZR'], ['LIQUID'], ['AL2SM'], [], [], [], [], [], []]


# Словарь с фазами

In [10]:
from pycalphad.core.utils import instantiate_models
instantiate_models(db, calc_components, getPhases(phases_mask))

{'AGSB_ORTHO': <pycalphad.model.Model at 0x2282ecfcd88>,
 'AL11SM3_HT': <pycalphad.model.Model at 0x22830df7f08>,
 'AL11SM3_LT': <pycalphad.model.Model at 0x2282e42b9c8>,
 'AL21V2': <pycalphad.model.Model at 0x22830df7188>,
 'AL23V4': <pycalphad.model.Model at 0x22830d1f2c8>,
 'AL2PD5': <pycalphad.model.Model at 0x2282ecdce48>,
 'AL2SM': <pycalphad.model.Model at 0x22830cf5dc8>,
 'AL2ZR': <pycalphad.model.Model at 0x2282ecd5d48>,
 'AL2ZR3': <pycalphad.model.Model at 0x22830ea05c8>,
 'AL3M_D022': <pycalphad.model.Model at 0x2282eea94c8>,
 'AL3NB': <pycalphad.model.Model at 0x2282ec7a1c8>,
 'AL3PD2_D513': <pycalphad.model.Model at 0x2282ecfc208>,
 'AL3SM': <pycalphad.model.Model at 0x2282fae3e88>,
 'AL3ZR': <pycalphad.model.Model at 0x2282ec6cdc8>,
 'AL3ZR2': <pycalphad.model.Model at 0x2282ec69cc8>,
 'AL3ZR5': <pycalphad.model.Model at 0x2282ecd5e08>,
 'AL45V7': <pycalphad.model.Model at 0x22830efce88>,
 'AL4RE_LT': <pycalphad.model.Model at 0x2282ec75cc8>,
 'AL4ZR5': <pycalphad.model.M

# Функции

In [11]:
from pycalphad.core.utils import instantiate_models
# instantiate_models(db, components, getAllPhases(components_phases))
print(getPhases(phases_mask))

['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 'RHOMBO_C19', 'SIGMA', 'TETRAGONAL_A6',

In [12]:
import re

# read the text file
text = ""
function_start = False
with open(db_name, 'r') as f:
    for line in f:
        if "Standard functions for the elements" in line:
            function_start = True
        if "Data for the elements from unary database" in line:
            function_start = False

        if function_start:
            text += line

component_functions = {}
# find all the component names in the text
component_names = re.findall(r'\$ ([A-Z][a-z]*)', text)
for component in component_names:
    # find the text between the component name and the next component name
    pattern = r'\$ {}.*?\$ [A-Z][a-z]*'.format(component)
    component_text = re.search(pattern, text, re.DOTALL).group()
    
    # find all the function names in the component text
    function_names = re.findall(r'FUNCTION ([A-Z]*)', component_text)
    
    component_functions[component] = function_names
component_functions = {k.upper():v for k,v in component_functions.items()}

In [13]:
# print the functions for each component
for component, functions in component_functions.items():
    if len(functions) > 0:
        print(f"{component} : {functions}")

AG : ['GHSERAG', 'GHCPAG', 'GBCCAG', 'GLIQAG', 'GDHCPAG']
AL : ['GHSERAL', 'GHCPAL', 'GBCCAL', 'GLIQAL', 'GLAVAL', 'GB', 'LB']
AM : ['GHSERAM', 'GFCCAM', 'GBCCAM', 'GLIQAM']
AS : ['GHSERAS', 'GLIQAS', 'GFCCAS', 'GBCCAS', 'GHCPAS']
AU : ['GHSERAU', 'GHCPAU', 'GBCCAU', 'GLIQAU']
B : ['GHSERB', 'GLIQB', 'GFCCB', 'GHCPB', 'GBCCB']
BA : ['GHSERBA', 'GLIQBA', 'GHCPBA', 'GFCCBA']
BE : ['GHSERBE', 'GBCCBE', 'GLIQBE', 'GFCCBE']
BI : ['GHSERBI', 'GLIQBI', 'GBCCBI', 'GFCCBI', 'GHCPBI']
C : ['GHSERC', 'GDIAC', 'GLIQC']
CA : ['GHSERCA', 'GBCCCA', 'GLIQCA', 'GHCPCA']
ADDED : ['GGASCA', 'GGASCA']
CD : ['GHSERCD', 'GLIQCD', 'GFCCCD', 'GBCCCD', 'GGASCD']
CE : ['GHSERCE', 'GBCCCE', 'GLIQCE', 'GHCPCE']
CO : ['GHSERCO', 'GFCCCO', 'GLIQCO', 'GBCCCO']
CR : ['GHSERCR', 'GLIQCR', 'GFCCCR', 'GHCPCR']
CS : ['GHSERCS', 'GLIQCS', 'GHCPCS', 'GFCCCS']
CU : ['GHSERCU', 'GBCCCU', 'GHCPCU', 'GLIQCU']
DY : ['GHSERDY', 'GBCCDY', 'GLIQDY', 'GFCCDY']
ER : ['GHSERER', 'GLIQER', 'GBCCER']
EU : ['GHSEREU', 'GLIQEU']
Z : ['GF

In [14]:
functions = set()
for component in components:
    if str(component) in component_functions.keys():
        functions.update(component_functions[component])
    else:
        print(f'Check - {component}')
print(functions)

Check - VA
{'GB', 'GBCCZR', 'GBCCCZ', 'GBCCAU', 'GLAVAL', 'GHCPAU', 'GBCCL', 'GHSERSE', 'GHSERAU', 'GBCCMZ', 'GFCCGD', 'GHSERAL', 'GHSERHG', 'GFE', 'GLIQSE', 'GBCCAL', 'GHSERGD', 'GBCCGD', 'GLIQGD', 'GHSERRH', 'GHCPRH', 'GHCPAL', 'GLIQZR', 'GHSERZR', 'GFCCHG', 'GHSERV', 'GLIQAL', 'GRHOMBHG', 'GFENI', 'GFCCZR', 'U', 'GHSERVZ', 'GLIQV', 'LB', 'GLIQRH', 'GHCPHG', 'GBCCRH', 'GLIQAU'}


In [15]:
from symengine import Piecewise, Symbol, lib
import math

temp = 1200
T = Symbol('T')
print(db.symbols['GBCCGD'])
print(type(temp))
expr = db.symbols['GBCCGD'].subs(T, temp)
print(expr)


Piecewise((-3600.78 + 95.01916415*T - 24.721413*T*log(T) - 8665.7*T**(-1.0) - 0.00285240521*T**2.0 - 3.14674076e-07*T**3.0, And(T < 1000.0, 100.0 <= T)), (152792.74 - 1349.588729*T + 180.097094*T*log(T) - 22038836.0*T**(-1.0) - 0.119550229*T**2.0 + 1.17915728e-05*T**3.0, And(T < 1508.15, 1000.0 <= T)), (-15783.76 + 202.2221768*T - 38.960425*T*log(T), And(T < 1586.15, 1508.15 <= T)), (-19850.56 + 224.8180291*T - 41.904333*T*log(T) + 995428.6*T**(-1.0) + 0.000858222759*T**2.0 - 3.77570269e-08*T**3.0, And(T < 3600.0, 1586.15 <= T)), (0, True))
<class 'int'>
Piecewise((105763.775257605 - 29665.6956*log(1200), False), (-1636855.92342827 + 216116.5128*log(1200), True), (226882.85216 - 46752.51*log(1200), False), (251931.19538381 - 50285.1996*log(1200), False), (0, True))


In [16]:
from collections import OrderedDict

function_value = OrderedDict()
for func in functions:
    expression = Piecewise()
    # print(func)
    if func not in db.symbols:
        print('Wrong function: ' + str(func))
        function_value[func] = None
        continue
    
    for arg in db.symbols[func].args:
        # condition = arg[0].subs(T, temp)
        # expression = arg[1].subs(T, temp)
        # result.append([condition, expression])
        if arg.is_Add:
            expression = arg.subs(T, temp).evalf()
        elif arg.is_Float or arg.is_Integer:
            expression = arg
        else:
            bool_res = False
            if len(arg.atoms()) > 0:
                bool_res = arg.subs(T, temp)
            if arg.is_Boolean:
                bool_res = arg
            if bool_res: 
                function_value[func] = expression
                break

print(function_value)

Wrong function: GB
Wrong function: GBCCL
Wrong function: GFE
Wrong function: GFENI
Wrong function: U
Wrong function: LB
OrderedDict([('GB', None), ('GBCCZR', -69104.2656226965), ('GBCCCZ', -5725.0), ('GBCCAU', 2930.0 + GHSERAU), ('GLAVAL', 5000.0 + GHSERAL), ('GHCPAU', 2160.75 + GHSERAU), ('GBCCL', None), ('GHSERSE', 0), ('GHSERAU', -77272.6856248379), ('GBCCMZ', 0.0), ('GFCCGD', 500.0 + GHSERGD), ('GHSERAL', -54976.3276402473), ('GHSERHG', -112210.200050902), ('GFE', None), ('GLIQSE', 0), ('GBCCAL', 4307.4 + GHSERAL), ('GHSERGD', -105270.77857148), ('GBCCGD', -104573.24219628), ('GLIQGD', -102198.298584263), ('GHSERRH', 0), ('GHCPRH', 2400.0 + GHSERRH), ('GHCPAL', 3321.0 + GHSERAL), ('GLIQZR', 7251.2987626752 + GHSERZR), ('GHSERZR', -68894.6050887857), ('GFCCHG', 10046.6 + GHSERHG), ('GHSERV', -57463.158455921), ('GLIQAL', -57996.2850123662), ('GRHOMBHG', -104149.498678239), ('GFENI', None), ('GFCCZR', 6520.0 + GHSERZR), ('U', None), ('GHSERVZ', -57463.158455921), ('GLIQV', 9415.59444

# Текущие результаты:

In [17]:
print('Список компонентов:')
print(components)

Список компонентов:
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE', 'VA']


In [18]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [19]:
print(f'Все фазы для компонентов из списка(бинарные): {bin(phases_mask)}')

Все фазы для компонентов из списка(бинарные): 0b1000010000000000000001000000000000000001100000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000001100000000000000011000000000000000000001000000000100000000000000000000000000000000000000000000010000000000000000000000000000000000000110000000000000000000000000000000000100000000000001000000000000000000000000000000000000000100000000000000000000000000000000000000000000001100000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000010000000000000000000000100000000000000000000000000111000000000000000000000000000000001100010000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000001000000100000001000000000000000000000000000000000110000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [20]:
print('Параметры расчета:')
print(f'Давление {P}, Температура {temp}, Количестов вещества по {amount} для компоненты')

Параметры расчета:
Давление 101325, Температура 1200, Количестов вещества по 0.125 для компоненты


In [21]:
print('Значения функций для компонентов:')
print(function_value)

Значения функций для компонентов:
OrderedDict([('GB', None), ('GBCCZR', -69104.2656226965), ('GBCCCZ', -5725.0), ('GBCCAU', 2930.0 + GHSERAU), ('GLAVAL', 5000.0 + GHSERAL), ('GHCPAU', 2160.75 + GHSERAU), ('GBCCL', None), ('GHSERSE', 0), ('GHSERAU', -77272.6856248379), ('GBCCMZ', 0.0), ('GFCCGD', 500.0 + GHSERGD), ('GHSERAL', -54976.3276402473), ('GHSERHG', -112210.200050902), ('GFE', None), ('GLIQSE', 0), ('GBCCAL', 4307.4 + GHSERAL), ('GHSERGD', -105270.77857148), ('GBCCGD', -104573.24219628), ('GLIQGD', -102198.298584263), ('GHSERRH', 0), ('GHCPRH', 2400.0 + GHSERRH), ('GHCPAL', 3321.0 + GHSERAL), ('GLIQZR', 7251.2987626752 + GHSERZR), ('GHSERZR', -68894.6050887857), ('GFCCHG', 10046.6 + GHSERHG), ('GHSERV', -57463.158455921), ('GLIQAL', -57996.2850123662), ('GRHOMBHG', -104149.498678239), ('GFENI', None), ('GFCCZR', 6520.0 + GHSERZR), ('U', None), ('GHSERVZ', -57463.158455921), ('GLIQV', 9415.59444185221 + GHSERV), ('LB', None), ('GLIQRH', -48315.478673478), ('GHCPHG', -103109.49867

In [22]:
print('Фазы для заданых параметров:')
print(eq_phases)
print('В бинарном виде:')
print([bin(phase) for phase in eq_phases_bin])

Фазы для заданых параметров:
['AUZR', 'LIQUID', 'AL2SM', '', '', '', '', '', '']
В бинарном виде:
['0b10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', '0b10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000